In [1]:
%load_ext autoreload

%autoreload 2

# Hourly
A simple hour tracker for git projects. ```hourly``` parses your commit messages for "clock in/out" keywords and uses their unix timestamps to precisely calculate work hours.


## Getting Started

## Install

    pip install hourly


### Requirements

	pandas
    gitpython

### Usage

Hourly will look for key words for clocking in/out.

To clock in:

    git commit -m "clock in - starting work on new feature"

do stuff as usual, then clock out

    git commit -m "clock out - finished feature"


# Tutorial

We can illustrate how to use hourly on the hourly repo itself.

    git clone https://github.com/asherp/hourly.git
    cd hourly

In [109]:
from hourly.hourly import get_work_commits, get_labor, get_earnings

```get_work_commits``` gathers all commits into a pandas array

In [123]:
work = get_work_commits('.')
work

,message,hash
time,,
2018-10-19 23:40:41-04:00,Initial commit,ef5690543bfb354b9325d1fbd1f9abbafbb4d9a4
2018-10-19 23:57:48-04:00,clock in\n,5c8f05b57b739ec525291c248ea9200651b49997
2018-10-20 00:21:40-04:00,preparing setup.py\n,254ecdacb52fc70bc358f8d55be58df3b70c7609
2018-10-20 00:39:11-04:00,clock out - work done for the day\n,0e33fa3d74f663f954b05dd9f30e0128ca7af162
2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,dc065b17337b14c2f8e0458de61e6880a338d6ae
2018-10-20 01:47:01-04:00,clock out\n,644ad6ebf4c9015fd512ed47b858602d784d6204
2018-10-20 01:47:45-04:00,clock in - pro bono\n,e6b5f78daa68e3731f82effccb66fd4bd14996bf
2018-10-20 01:51:36-04:00,clock out - pro bono\n,1aff88af5e9688645966ccd15da8e1530205cfea
2018-10-20 02:03:56-04:00,clock in - finishing tutorial\n,53bd7316e579d8582c46af09277b40fbba3a390e


In [ ]:
    work.index = [pd.Timestamp(i) for i in work.index]
    work.index = pd.DatetimeIndex(work.index, utc = True)

In [126]:
pd.DatetimeIndex(pd.Timestamp(i) for i in work.index)

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

```get_labor``` calculates hours worked by differencing commit timestamps. 
*Note: ```get_labor``` raises an error if clock in and clock out are of different lengths *

### Getting time card

In [111]:
import pandas as pd
pd.set_option('display.width', 400)

In [117]:
pd.DatetimeIndex(work.index)

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [114]:
work.in

Index([2018-10-19 23:40:41-04:00, 2018-10-19 23:57:48-04:00, 2018-10-20 00:21:40-04:00, 2018-10-20 00:39:11-04:00, 2018-10-20 01:06:08-04:00, 2018-10-20 01:47:01-04:00, 2018-10-20 01:47:45-04:00, 2018-10-20 01:51:36-04:00, 2018-10-20 02:03:56-04:00, 2018-10-20 02:11:54-04:00, 2018-10-20 02:14:21-04:00, 2018-10-20 02:18:27-04:00, 2018-10-20 02:22:56-04:00, 2018-10-20 02:51:13-04:00,
       2018-10-20 02:57:22-04:00, 2018-10-20 11:53:00-04:00],
      dtype='object')

In [113]:
get_labor(work, end_date='2018-10-20 02:13:54')

TypeError: Cannot compare tz-naive and tz-aware timestamps

### Handling errant clock in/out messages
If you mistakenly put "clock out" in a message, hourly will interpret the message as a legitimate end time. This will likely raise an error when computing the labor. For example, here is what happens when we include a problematic commit in the labor calculation

In [87]:
work[work.hash == 'd9ec537b36475b565df6b28d0cab6edc3a89f2da']

,message,hash
time,,
2018-10-20 02:14:21-04:00,had to clock out so notebook examples don't br...,d9ec537b36475b565df6b28d0cab6edc3a89f2da


In [83]:
errant_clocks = ['d9ec537b36475b565df6b28d0cab6edc3a89f2da']

### Filtering work session keywords

Use the "ignore" key word to skip any work you don't want to include in your invoices.

In [9]:
labor = get_labor(work, ignore = 'pro bono')
labor

pay period: 2018-10-19 23:57:48-04:00 -> 2018-10-20 02:11:54-04:00
ignoring pro bono


,TimeIn,log in,TimeOut,log out,TimeDelta
0,2018-10-19 23:57:48-04:00,clock in\n,2018-10-20 00:39:11-04:00,clock out - work done for the day\n,00:41:23
1,2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,2018-10-20 01:47:01-04:00,clock out\n,00:40:53
3,2018-10-20 02:03:56-04:00,clock in - finishing tutorial\n,2018-10-20 02:11:54-04:00,clock out - converted notebook for README\n,00:07:58


## Get total earnings

In [17]:
get_earnings(labor.TimeDelta)

0 days 01:30:14, 1.5038888888888888 hours worked
120.31 usd


120.31